# Open this notebook in colab
#<a href="https://colab.research.google.com/github/da-roth/NeuronalNetworkTensorflowFramework/blob/main/src/Examples_Colab/MonteCarloLearning/Implementation_Multilevel_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%%capture
pip install git+https://github.com/da-roth/NeuronalNetworkTensorflowFramework#montecarlolearning

In [ ]:
###
### 0. Import packages and references
###
### - dataSeed = seed for simulations or (for csv input) for randomization of csv

from montecarlolearning import *

###
### 1. Set Generator: Single-level with Euler-Maruyama and h = 2^(-4) step-width	
###
Generator = Multilevel_GBM(Multilevel_Train_Case.Euler, 4)
Generator.set_inputName('S')
Generator.set_outputName('EuropeanCallPrice(S)')

###
### 2. Set Nueral network structure / 
# Hyperparameters
### 

Regressor = Neural_Approximator()
Regressor.set_Generator(Generator)
Regressor.set_hiddenNeurons(20)
Regressor.set_hiddenLayers(2)
Regressor.set_activationFunctionsHidden(tf.nn.sigmoid)
Regressor.set_activationFunctionOutput(tf.nn.sigmoid)
Regressor.set_weight_seed(1)

TrainSettings = TrainingSettings()
TrainSettings.useExponentialDecay(0.01, 0.1, 1000)
TrainSettings.set_min_batch_size(1)
TrainSettings.set_test_frequency(500)
TrainSettings.set_nTest(10000)
TrainSettings.set_samplesPerStep(10000)
TrainSettings.set_trainingSteps(1000)

###
### 3. Train network and Study results
### Comment: For different trainingSetSizes the neural network reset and not saved, hence train and evaluation of yPredicted are done together currently
###

xTest, yTest, yPredicted = train_and_test(Generator, Regressor, TrainSettings)
plot_results("predicted vs. expected", yPredicted, xTest, yTest, Generator)

In [ ]:
###
### Multi-level Monte Carlo Learning 
###

# Train level 0 with Milstein and h = 2^(0) step-width
Generator = Multilevel_GBM(Multilevel_Train_Case.Milstein, 0)
Generator.set_inputName('S')
Generator.set_outputName('MultiLevelApproximation')
# Train level 1 (level estimator) with Milstein and h_fine = 2^(-1) and h_coarse = 2^(0)
GeneratorLevel1 = Multilevel_GBM(Multilevel_Train_Case.LevelEstimator, 0)
GeneratorLevel1 .set_inputName('x')
GeneratorLevel1 .set_outputName('P_1-P_0')
# Train level 2 (level estimator) with Milstein and h_fine = 2^(-2) and h_coarse = 2^(-1)
GeneratorLevel2 = Multilevel_GBM(Multilevel_Train_Case.LevelEstimator, 1)
GeneratorLevel2 .set_inputName('x')
GeneratorLevel2 .set_outputName('P_2-P_1')
# Train level 3 (level estimator) with Milstein and h_fine = 2^(-3) and h_coarse = 2^(-2)
GeneratorLevel3 = Multilevel_GBM(Multilevel_Train_Case.LevelEstimator, 2)
GeneratorLevel3 .set_inputName('x')
GeneratorLevel3 .set_outputName('P_3-P_2')
# Train level 4 (level estimator) with Milstein and h_fine = 2^(-4) and h_coarse = 2^(-3)
GeneratorLevel4 = Multilevel_GBM(Multilevel_Train_Case.LevelEstimator, 3)
GeneratorLevel4 .set_inputName('x')
GeneratorLevel4 .set_outputName('P_4-P_3')


###
### 2. Set Nueral network structure / Hyperparameters
### 

Regressor = Neural_Approximator()
Regressor.set_Generator(Generator)
Regressor.set_hiddenNeurons(20)
Regressor.set_hiddenLayers(2)
Regressor.set_activationFunctionsHidden(tf.nn.sigmoid)
Regressor.set_activationFunctionOutput(tf.nn.sigmoid)
Regressor.set_weight_seed(1)

TrainSettings = TrainingSettings()
TrainSettings.useExponentialDecay(0.01, 0.1, 1000)
TrainSettings.set_min_batch_size(1)
TrainSettings.set_test_frequency(500)
TrainSettings.set_nTest(10000)
TrainSettings.set_samplesPerStep(40000)
TrainSettings.set_trainingSteps(1000)

###
### 3. Train and test
### Comment: For different trainingSetSizes the neural network reset and not saved, hence train and evaluation of yPredicted are done together currently
###

xTest, yTest, yPredicted = train_and_test(Generator, Regressor, TrainSettings)
TrainSettings.set_samplesPerStep(10000)
TrainSettings.useExponentialDecay(0.01, 0.1, 500)
xTest2, yTest2, yPredictedLevel1 = train_and_test(GeneratorLevel1, Regressor, TrainSettings)
TrainSettings.set_samplesPerStep(10000)
xTest, yTest, yPredictedLevel2 = train_and_test(GeneratorLevel2, Regressor, TrainSettings)
TrainSettings.set_samplesPerStep(10000)
xTest, yTest, yPredictedLevel3 = train_and_test(GeneratorLevel3, Regressor, TrainSettings)
xTest, yTest, yPredictedLevel4 = train_and_test(GeneratorLevel4, Regressor, TrainSettings)

# Add all levels for test data
multilevel_predicted = {('standard', 10000): np.add(yPredicted[('standard', 10000)], yPredictedLevel1[('standard', 10000)])}
multilevel_predicted = {('standard', 10000): np.add(multilevel_predicted[('standard', 10000)], yPredictedLevel2[('standard', 10000)])}
multilevel_predicted = {('standard', 10000): np.add(multilevel_predicted[('standard', 10000)], yPredictedLevel3[('standard', 10000)])}
multilevel_predicted = {('standard', 10000): np.add(multilevel_predicted[('standard', 10000)], yPredictedLevel4[('standard', 10000)])}

# Plot P_0 + [P_1-P_0] + [P_2-P_1] + [P_3-P_2] + [P_4-P_3]
plot_results("predicted vs. expected", multilevel_predicted, xTest, yTest, Generator)